# Plots zum Klimawandel

## Global: Durchschnittstemperatur (Land und Ozean zusammen)

Wir wollen nun die globalen Temperaturen visualisieren und greifen dazu auf die Daten der [NASA](https://data.giss.nasa.gov/gistemp/graphs_v4/) zurück.

Es werden somit Daten von Dateien (online oder offline) eingelesen mit der Python Bilbiothek `pandas`

### Daten importieren

In [ ]:
import pandas as pd
link = "https://data.giss.nasa.gov/gistemp/graphs_v4/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.csv"

Temp_NASA = pd.read_csv(link, header=1) # einlesen
Temp_NASA # Ausgabe

Die Spalten können nun über den Namen des DataFrames und die jeweilige Spalte in eckigen Klammern mit Hochkommas verwendet werden, z.B.:

In [ ]:
Temp_NASA["Year"]

### Daten plotten

**Aufgabe a)** Plotten Sie die Spalte `No_Smoothing` (Werte) und `Lowess(5)` (Glättung) über die (Jahre) mit der Spalte `Year`

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('default')
plt.figure(figsize=(10,4))
plt.rcParams['font.size'] = 14;
plt.ylabel("Jahresmitteltemperaturabweichung [°C]")
plt.plot(Temp_NASA["Year"],Temp_NASA["No_Smoothing"], ls="-", lw=1, marker="s", ms=3, color="tab:blue", alpha=0.5, label="Werte");
plt.plot(Temp_NASA["Year"],Temp_NASA["Lowess(5)"], lw=3,  color="tab:blue", label="geglättet");
plt.legend();
plt.grid();

### Datenverarbeitung

Die Bibliothek `pandas` ist sehr umfangreich und wird viel zur Datenverarbeitung genutzt. Im folgenden dazu einige Beispiele

**alle Spalten analysieren (Anzahl/Mittelwert/Min/Max/Percentile)**

In [ ]:
Temp_NASA.describe()

**Maximum bestimmen** (Beispiel: stärkster Tempetaturunterschied)

In [ ]:
Temp_NASA["No_Smoothing"].max()

**index zum Maximum bestimmen** (Beispiel: Jahr des stärksten Tempetaturunterschieds)

- Es wird ein `index` (entspricht meistens Zeilennummer) ausgegeben

In [ ]:
index_max = Temp_NASA["No_Smoothing"].idxmax()
print(index_max)

mit dem `.loc` Befehl wird die Tabelle mit nun gefunden `index` ausgegeben werden

In [ ]:
Temp_NASA.loc[index_max]

Mit `.sort_values("Spaltenname")` können wir auch die Tabelle nach dem definierten Spaltennamen sortieren (standardmäßig in **ansteigender** Reihenfolge)

In [ ]:
Temp_NASA.sort_values("No_Smoothing")

mit `ascending=False` wird absteigend sortiert

In [ ]:
Temp_NASA.sort_values("No_Smoothing",ascending=False)

mit `.head(Anzahl)` können wir angeben wieviele Zeilen (vom **Kopf** beginnend) **angezeigt** werden. Mit `.tail(Anzahl)` erfolgt die ausgabe der letzten Werte mit der jeweiligen Anzahl

**WICHTIG**: Der Dataframe `Temp_NASA` wird dadurch nicht verändert, es handelt sich nur um eine Anzeige!

In [ ]:
Temp_NASA.sort_values("No_Smoothing",ascending=False).head(10)

**Aufgabe**: Wie viele 10 stärksten (positiven) Abweichungen lagen in den letzten 10 Jahren?

Ein möglicher Weg:
1. Neue Dataframe `top10` erzeugen mit den 10 stärksten positiven Abweichungen
2. Prüfen welche Jahre davor in den letzten 10 Jahren sind und dies als neue Spalte `last 10 years?` speichern
3. Ausgabe des Dataframes mit der neuen Spalte

In [ ]:
top10 = Temp_NASA.sort_values("No_Smoothing",ascending=False)[0:10] # mit [0:10] nehmen wir Spalten von 0 bis ausschließlich 10 (Anzahl=10)
top10["last 10 years?"] = top10["Year"]>=(2021-10) # Neue Spalte "last 10 years?" erstellt > geprüft ob die Spalte "Year" Einträge hat die größer gleich 2021-10 sind
top10 #Ausgabe

In den NASA Originaldaten gibt es bereits eine Glättung. Diese können wir z.B. durch einen rollenden Mittelwert selbst erzeugen.

Dazu benutzen wir die Funktion `.rolling(Anzahl, center=True).mean()` (mit `center=true` wird das Label auf die Mitte gesetzt)

**Wichtig**: Die Tabelle muss in dem Fall nach den Jahren sortiert sein, deshalb prüfen wir dies erneut:

In [ ]:
Temp_NASA

In [ ]:
Temp_NASA["Rollender Mittelwert (5)"] = Temp_NASA["No_Smoothing"].rolling(5,center=True).mean()
Temp_NASA

Eine weitere Variante der Glättung ist die **LO**cally **WE**ighted **S**catter-plot **S**moother (LOWESS). Diese wurde scheinbar auch von der NASA verwendet. Eine genauere Erklärung zur Methode findet ihr [hier](https://youtu.be/b7oryuMP3r8).

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess
Temp_NASA["Lowess(own)"] = lowess(Temp_NASA["No_Smoothing"],Temp_NASA["Year"], frac=1/14)[:,1]

In [ ]:
Temp_NASA

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('default')

plt.figure(figsize=(10,4))
plt.rcParams['font.size'] = 14;
plt.ylabel("Jahresmitteltemperaturabweichung [°C]")
plt.plot(Temp_NASA["Year"],Temp_NASA["No_Smoothing"], ls="-", lw=1, marker="s", ms=3, color="tab:gray", alpha=0.5, label="Werte");
plt.plot(Temp_NASA["Year"],Temp_NASA["Lowess(5)"], lw=4,  color="tab:blue", label="Glättung (NASA)");
plt.plot(Temp_NASA["Year"],Temp_NASA["Rollender Mittelwert (5)"], lw=2,  color="tab:red", label="eigene Glättung (Rollender Mittelwert 5 , zentriert)");
plt.plot(Temp_NASA["Year"],Temp_NASA["Lowess(own)"],ls="--",lw=2,  color="orange", label="eigene Glättung (LOWESS f=1/14)");

plt.legend();
plt.grid();

Die Verwendung des rollenden Mittelwertes hat das Problem, dass dabei immer Werte fehlen (Anfang/Ende). Die gewählten Mit den gewählten Parametern für das LOWESS sind die Ergebnisse ähnlich zur Original Glättung

## Global: Durchschnittstemperatur (Land + Ozean getrennt)

**Aufgabe b)** Stellen Sie die Kurven für die Erwärmung an Landoberfläche und an der Ozeanoberfläche grafisch dar und beschriften Sie das Diagramm entsprechend

### Daten importieren

In [ ]:
link = "https://data.giss.nasa.gov/gistemp/graphs_v4/graph_data/Temperature_Anomalies_over_Land_and_over_Ocean/graph.csv"

In [ ]:
Temp_NASA2 = pd.read_csv(link, header=1) # einlesen
Temp_NASA2 # Ausgabe

### Daten plotten

In [ ]:
plt.style.use('default')

plt.figure(figsize=(10,4))
plt.rcParams['font.size'] = 14;
plt.ylabel("Jahresmitteltemperaturabweichung [°C]")
plt.plot(Temp_NASA2["Year"],Temp_NASA2["Land_Annual"], ls="-", lw=1, marker="s", ms=3, color="tab:red", alpha=0.5, label="Land");
plt.plot(Temp_NASA2["Year"],Temp_NASA2["Lowess(5)"], lw=3,  color="tab:red", label="Land (geglättet)");
plt.plot(Temp_NASA2["Year"],Temp_NASA2["Ocean_Annual"], ls="-", lw=1, marker="s", ms=3, color="tab:blue", alpha=0.5, label="Ozean");
plt.plot(Temp_NASA2["Year"],Temp_NASA2["Lowess(5).1"], lw=3,  color="tab:blue", label="Ozean (geglättet)");
plt.legend();
plt.grid();

## Deutschland: Durchschnittstemperatur (Land)

Beim [Umweltbundesamt](https://www.umweltbundesamt.de/daten/klima/trends-der-lufttemperatur#2020-bisher-zweitwarmstes-jahr-in-deutschland) gibt es aktuelle Zahlen zur Erwärmung der Luftemperatur in Deutschland. Die Werte befinden sich in [dieser Exceltabelle](https://www.umweltbundesamt.de/sites/default/files/medien/384/bilder/dateien/3-bis-8_abb-tab_tmt_2021-05-12.xlsx), welche mit dem Befehl `pd.read_excel()` eingelesen werden kann, wobei hierbei der Name des ExcelSheets und die verwendenten Spaltennummern angegeben werden können.

### Daten importieren

In [ ]:
# Einlesen
link = "https://www.umweltbundesamt.de/sites/default/files/medien/384/bilder/dateien/3-bis-8_abb-tab_tmt_2021-05-12.xlsx"

Temp_GER = pd.read_excel(link,sheet_name="3_DWD",usecols=[1,19]) # einlesen
Temp_GER.columns=["Jahr","Jahresmitteltemperatur [°C]"]
Temp_GER.to_csv("data/Temp_Deutschland.csv", index=False)

### Daten aufbereiten

In [ ]:
# Daten aufbereiten
Temp_GER.dropna(inplace=True) # Zeilen mit "NaN" Werte rausschmeissen und dataframe speichern 
Temp_GER["Jahr"] = Temp_GER["Jahr"].astype(int) # Spalte "Jahr" als integer definieren
Temp_GER

In [ ]:
# Temperaturabweichung berechnen (Abweichung von Mittelwert der Temperatur zwischen 1881 und 1900)
mean_GER1881_1910 = Temp_GER.loc[Temp_GER["Jahr"] <= 1910,"Jahresmitteltemperatur [°C]"].mean()
Temp_GER["Jahresmitteltemperaturabweichung [°C]"] = Temp_GER["Jahresmitteltemperatur [°C]"] - mean_GER1881_1910
Temp_GER

In [ ]:
# Glättung
from statsmodels.nonparametric.smoothers_lowess import lowess
Temp_GER["Lowess(own)"] = lowess(Temp_GER["Jahresmitteltemperaturabweichung [°C]"],Temp_GER["Jahr"], frac=1/14)[:,1]
Temp_GER

### Daten plotten

**Aufgabe c)** Stellen Sie folgende Daten zusammen dar:
- global: Land+Ozean (geglättet) <- NASA
- global: Land (geglättet) <- NASA
- Deutschland: Land (Werte) <- DWD
- Deutschland: Land (geglättet) <- DWD

In [ ]:
plt.style.use('default')

plt.figure(figsize=(10,4))
plt.rcParams['font.size'] = 14;
plt.ylabel("Jahresmitteltemperaturabweichung [°C]")
plt.plot(Temp_NASA["Year"],Temp_NASA["Lowess(5)"], lw=3,  color="tab:blue", label="global: Land+Ozean (5 Jahre geglättet) [NASA]");
plt.plot(Temp_NASA2["Year"],Temp_NASA2["Lowess(5)"], lw=3,  color="tab:red", label="global: Land (5 Jahre geglättet) [NASA]");
plt.plot(Temp_GER["Jahr"],Temp_GER["Jahresmitteltemperaturabweichung [°C]"], ls="-", lw=1,marker="s", ms=3, color="gray",alpha=0.5, label="Deutschland Land (Werte) [DWD]");
plt.plot(Temp_GER["Jahr"],Temp_GER["Lowess(own)"], ls="-", lw=2, color="black", label="Deutschland Land (LOWESS) [DWD]");

plt.legend(fontsize=12);
plt.grid();

### Vergleich mit Abbildung aus Leoplodina Factsheet

Vergleichen wir diese Darstellung nun mal mit einer aus dem [Factsheet der Leopoldina](https://doi.org/10.26164/leopoldina_03_00327):

Nationale Akademie der Wissenschaften Leopoldina (2021): Klimawandel: Ursachen,
Folgen und Handlungsmöglichkeiten. Halle (Saale).

![](images/FactSheetLeopoldina_TempGer.png)

Hierbei wurde scheinbar eine sehr starke Glättung der Messdaten verwendet, wir erhöhen also den Wert `frac` für die LOWESS Methode und prüfen ob wir in ähnliche Bereiche kommen

In [ ]:
# Glättung
from statsmodels.nonparametric.smoothers_lowess import lowess
Temp_GER["Lowess(own2)"] = lowess(Temp_GER["Jahresmitteltemperaturabweichung [°C]"],Temp_GER["Jahr"], frac=1/3)[:,1]
Temp_GER_Lowess_own2_mean_untill_1910 = Temp_GER.loc[Temp_GER["Jahr"] <= 1910,"Lowess(own2)"].mean()
Temp_GER["Lowess(own2)"] = Temp_GER["Lowess(own2)"] - Temp_GER_Lowess_own2_mean_untill_1910

Temp_NASA["Lowess(own3)"] = lowess(Temp_NASA["No_Smoothing"],Temp_NASA["Year"], frac=1/3)[:,1]
Temp_NASA_Lowess_own3_mean_untill_1910 = Temp_NASA.loc[Temp_NASA["Year"] <= 1910,"Lowess(own3)"].mean()
Temp_NASA["Lowess(own3)"] = Temp_NASA["Lowess(own3)"] - Temp_NASA_Lowess_own3_mean_untill_1910


plt.style.use('default')

plt.figure(figsize=(10,4))
plt.rcParams['font.size'] = 14;
plt.ylabel("Jahresmitteltemperaturabweichung [°C]")
plt.plot(Temp_NASA["Year"],Temp_NASA["Lowess(own3)"], lw=3,  color="tab:red", label="global: Land+Ozean (eigene Glättung) [NASA]");
plt.plot(Temp_GER["Jahr"],Temp_GER["Lowess(own2)"], ls="-", lw=2, color="gray", label="Deutschland Land (LOWESS) [DWD]");

plt.legend(fontsize=12);
plt.grid();

Ausgabe der Temperaturdifferenz im Jahr 2020

In [ ]:
Temp_NASA.loc[Temp_NASA["Year"] == 2020]

In [ ]:
Temp_GER.loc[Temp_GER["Jahr"] == 2020]

**Zusatzaufgabe** : Bestimmen Sie den linearen Anstieg für die Temperaturwerte  von Deutschland (LOWESS mit frac=1)

In [ ]:
import numpy as np
import pandas as pd

x=Temp_GER["Jahr"]
y=Temp_GER["Jahresmitteltemperatur [°C]"]
model = np.polyfit(x, y, 1) # 1. Wert = Anstieg , 2. Wert = Schnittpunkt mit y-Achse
y_model = model[0]*x+model[1]

plt.style.use('default')
plt.figure(figsize=(10,4))
plt.rcParams['font.size'] = 14;
plt.ylabel("Jahresmitteltemperaturabweichung [°C]")
plt.plot(Temp_GER["Jahr"],Temp_GER["Jahresmitteltemperatur [°C]"], ls="-", lw=1,marker="s", ms=3, color="gray",alpha=0.5, label="Deutschland Land (Werte) [DWD]");
plt.plot(x,y_model, ls="-", lw=3, color="tab:blue",alpha=0.5, label=f"lineare Regression (y={model[0]:.3f}*x+{model[1]:.3f})");
plt.legend(fontsize=12);
plt.grid();

In [ ]:
print(f"Temperaturanstieg pro Jahr (von 1981 bis 2020): {model[0]:.3f}°C/Jahr")
print(f"Temperaturanstieg seit Beginn der Messung: {(y_model.iloc[-1]-y_model.iloc[0]):.3f}°C")

**Vorsicht**: Die lineare Regression bezieht hier den ganzen Zeitraum mit ein, betrachten wir für den Temperaturgradienten nur die Daten von 1980 bis 2020:

In [ ]:
x=Temp_GER.loc[Temp_GER["Jahr"] >= 1980,"Jahr"]
y=Temp_GER.loc[Temp_GER["Jahr"] >= 1980,"Jahresmitteltemperatur [°C]"]
model = np.polyfit(x, y, 1) # 1. Wert = Anstieg , 2. Wert = Schnittpunkt mit y-Achse
y_model = model[0]*x+model[1]
print(f"Temperaturanstieg pro Jahr (von 1980 bis 2020): {model[0]:.3f}°C/Jahr")

In [ ]:
plt.style.use('default')
plt.figure(figsize=(10,4))
plt.rcParams['font.size'] = 14;
plt.ylabel("Jahresmitteltemperaturabweichung [°C]")
plt.plot(Temp_GER["Jahr"],Temp_GER["Jahresmitteltemperatur [°C]"], ls="-", lw=1,marker="s", ms=3, color="gray",alpha=0.5, label="Deutschland Land (Werte) [DWD]");
plt.plot(x,y_model, ls="-", lw=3, color="tab:blue",alpha=0.5, label=f"lineare Regression (y={model[0]:.3f}*x+{model[1]:.3f})");
plt.legend(fontsize=12);
plt.grid();

## Treibhausgasemissionen

Zunächst stellen wir die Treibhausgas-Konzentrationen dar <br>
Quelle : [Umweltbundesamt](https://www.umweltbundesamt.de/daten/klima/atmosphaerische-treibhausgas-konzentrationen#beitrag-langlebiger-treibhausgase-zum-treibhauseffekt)

### Daten imortieren

In [ ]:
link = "https://www.umweltbundesamt.de/sites/default/files/medien/384/bilder/dateien/6_abb_treibhausgas-gesamt-konz_2020-06-03.xlsx"

THG_Emission_UBA = pd.read_excel(link,sheet_name="Daten",usecols=[1,2],header=8) # einlesen
THG_Emission_UBA.to_csv("data/THG_Emission_UBA.csv")

### Daten Plotten

**Aufgabe d)** Stellen Sie die Treibhausgasemissionen über die Jahre dar

In [ ]:
plt.style.use('default')

plt.figure(figsize=(10,4))
plt.rcParams['font.size'] = 14;
plt.ylabel("CO2-Äquivalent [ppm]")
plt.plot(THG_Emission_UBA["Jahr"],THG_Emission_UBA["CO2-eq\n(ppm)"], lw=3,  color="tab:red");
plt.grid();

## Vergleich zweier CO2 Messtationen 

Auf der Seite vom Umweltbundesamt findet man auch CO2 Messwerte verschiedener Messtationen: <br>
Quelle : [Umweltbundesamt](https://www.umweltbundesamt.de/daten/klima/atmosphaerische-treibhausgas-konzentrationen#kohlendioxid-)

[Quelle Welttrend WMO](https://gaw.kishou.go.jp/publications/global_mean_mole_fractions#content2) <br>
[Quelle Mauna Loa](https://www.esrl.noaa.gov/gmd/ccgg/trends/data.html) <br>
[Quelle Schauinsland  und Zugspitze](http://www.uba.de/gaw)

In [ ]:
link="https://www.umweltbundesamt.de/sites/default/files/medien/384/bilder/dateien/2-4_abb_langzeitreihen-konz_2021-05-26.xlsx"

CO2_Emission_UBA = pd.read_excel(link,sheet_name="Kohlendioxid-Daten",usecols=[0,1,2,3,4]) # einlesen
CO2_Emission_UBA.columns = ["Jahr","Mauna Loa, Hawaii","Schauinsland","Zugspitze","Welttrend WMO"]
CO2_Emission_UBA.to_csv("data/CO2_Emission_UBA.csv")
CO2_Emission_UBA.head()


In [ ]:
plt.style.use('default')

plt.figure(figsize=(10,4))
plt.rcParams['font.size'] = 14;
plt.ylabel("$\mathrm{CO_2}$-Konzentration in der Atmosphäre [ppm]")
plt.plot(CO2_Emission_UBA["Jahr"],CO2_Emission_UBA["Mauna Loa, Hawaii"], lw=2,  label="Mauna Loa, Hawaii", c="tab:orange");
plt.plot(CO2_Emission_UBA["Jahr"],CO2_Emission_UBA["Schauinsland"], lw=2,  label="Schauinsland", c="tab:green");
plt.plot(CO2_Emission_UBA["Jahr"],CO2_Emission_UBA["Zugspitze"], lw=2,  label="Zugspitze", c="tab:blue");
plt.plot(CO2_Emission_UBA["Jahr"],CO2_Emission_UBA["Welttrend WMO"], lw=2,  label="Welttrend WMO", c="tab:red");

plt.legend();
plt.grid();

## Übersicht Treibhausgase

[Quelle: Umweltbundesamt](https://www.umweltbundesamt.de/daten/klima/atmosphaerische-treibhausgas-konzentrationen#beitrag-langlebiger-treibhausgase-zum-treibhauseffekt)

Excel-Tabelle: https://www.umweltbundesamt.de/sites/default/files/medien/384/bilder/dateien/5_abb_beitrag-treibhauseffekt-co2-thg_2021-05-26.xlsx

Quelle Daten: NOAA Earth System Research Laboratory, The NOAA annual greenhouse gas index (AGGI), https://www.esrl.noaa.gov/gmd/aggi/aggi.html

In [ ]:
# Daten
Beitrag = np.array([2076,516,202,161,57,129])
Beitrag_Prozent = Beitrag/Beitrag.sum()*100
Bezeichnung = np.array(["Kohlendioxid (CO₂)","Methan (CH₄)","Lachgas (N₂O)","Dichlordifluormethan (CFC-12)","Trichlorfluormethan (CFC-11)","15 weitere Treibhausgase"])

# Plot
plt.figure(figsize=(6,4));
plt.rcParams['font.size'] = 14;
plt.bar(Bezeichnung,Beitrag_Prozent);
plt.ylabel('Anteil [%]');
plt.title('Beitrag zum Treibhauseffekt durch verschiedene Treibhausgase', fontsize=12);
plt.xticks(rotation=30, ha='right', fontsize=12);
plt.grid(axis='y');
plt.gca().set_axisbelow(True);

In [ ]:
# Daten
Beitrag = np.array([2076,516,202,161,57,129])
Beitrag_Prozent = Beitrag/Beitrag.sum()*100
Bezeichnung = np.array(["Kohlendioxid (CO₂)","Methan (CH₄)","Lachgas (N₂O)","Dichlordifluormethan (CFC-12)","Trichlorfluormethan (CFC-11)","15 weitere Treibhausgase"])

# Plot
plt.figure(figsize=(6,4))
plt.rcParams['font.size'] = 14;
plt.barh(Bezeichnung,Beitrag_Prozent)
plt.ylabel('Anteil [%]')
plt.title('Beitrag zum Treibhauseffekt durch verschiedene Treibhausgase', fontsize=12);
plt.grid(axis='x');
plt.gca().set_axisbelow(True);

Wenn wir die Einträge sortieren möchten, müssen wir die Listen in ein `pandas` `dataframe` einfügen

In [ ]:
THG_Anteil = pd.DataFrame(data=([Bezeichnung,Beitrag,Beitrag_Prozent])).T
THG_Anteil.columns=["Name","Anteil","Anteil [%]"]
THG_Anteil

In [ ]:
THG_Anteil.sort_values("Anteil [%]",ascending=False, inplace=True)
THG_Anteil

In [ ]:
# Plot
plt.figure(figsize=(6,4))
plt.rcParams['font.size'] = 14;
plt.barh(THG_Anteil["Name"],THG_Anteil["Anteil [%]"])
plt.ylabel('Anteil [%]')
plt.title('Beitrag zum Treibhauseffekt durch verschiedene Treibhausgase', fontsize=12);
plt.grid(axis='x');
plt.gca().set_axisbelow(True);